In [121]:
from geopy import Location, Nominatim
from ipyleaflet import AwesomeIcon, Map, Marker, Polyline
from routingpy import Valhalla

# Search for a start location

Uses **geopy** to search **Nominatim**

In [104]:
# Don't abuse

# geolocator = Nominatim(user_agent="chris_test")
# tri = geolocator.geocode("Toyota Research Institute")

In [107]:
# Saved result for TRI HQ
tri = Location(
    "Toyota Research Institute, 4440, West El Camino Real, Los Altos, Santa Clara County, California, 94022, United States", 
    (37.40253645, -122.1165510679842, 0.0),
    {
        'place_id': 311529403,
         'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright',
         'osm_type': 'way',
         'osm_id': 256656808,
         'lat': '37.40253645',
         'lon': '-122.1165510679842',
         'class': 'building',
         'type': 'commercial',
         'place_rank': 30,
         'importance': 9.99999999995449e-06,
         'addresstype': 'building',
         'name': 'Toyota Research Institute',
         'display_name': 'Toyota Research Institute, 4440, West El Camino Real, Los Altos, Santa Clara County, California, 94022, United States',
         'boundingbox': ['37.4021493', '37.4029503', '-122.1171370', '-122.1159999']
    }
)

# Pick an end location

Show start and end marker with **ipyleaflet**.
Drag the end marker as desired

In [123]:
center = (tri.latitude, tri.longitude)
tri_map = Map(center=center, zoom=16)

startpoint_icon = AwesomeIcon(
    name='home',
    marker_color='green',
    icon_color='black',
    spin=False
)

endpoint_icon = AwesomeIcon(
    name='flag-checkered',
    marker_color='red',
    icon_color='black',
    spin=False
)


startpoint_marker = Marker(location=center, draggable=False, icon=startpoint_icon)
tri_map.add_control(startpoint_marker)
endpoint_marker = Marker(location=center, draggable=True, icon=endpoint_icon)
tri_map.add_control(endpoint_marker)
    
display(tri_map)

Map(center=[37.40253645, -122.1165510679842], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom…

# Get Directions

Uses **routingpy** againsts a locally running **Valhalla**

```bash
docker run --rm --name valhalla_gis-ops -p 8002:8002 -v $PWD/custom_files:/custom_files -e tile_urls=https://download.geofabrik.de/north-america/us/california/norcal-latest.osm.pbf ghcr.io/gis-ops/docker-valhalla/valhalla:latest
```

In [136]:
start = tri.longitude, tri.latitude
end = endpoint_marker.location[1], endpoint_marker.location[0]
print("Getting directions for", start, end)

MPH_TO_KPH = 1.60934

directions = Valhalla('http://localhost:8002').directions(
    locations=[start, end], 
    profile="pedestrian",
    units="mi",
    options={
        "walking_speed": 2.5 * MPH_TO_KPH,
    }
)
print("Duration (minutes):", directions.duration / 60)

Getting directions for (-122.1165510679842, 37.40253645) (-122.11183547973634, 37.40173282937721)
Duration (minutes): 8.016666666666667


In [137]:
locations = [(lat, long) for (long, lat) in directions.geometry]
line = Polyline(
    locations=locations,
    color="green" ,
    fill=False
)

tri_map.add(line)
display(tri_map)

Map(bottom=6506845.0, center=[37.40253645, -122.1165510679842], controls=(ZoomControl(options=['position', 'zo…